In [191]:
from autoscraper import AutoScraper

In [204]:
amazon_url="https://www.amazon.com/s?k=playstation+4&ref=nb_sb_noss_2"

wanted_list=["₹599.00 ","Electric kettle","18.5L x 18.5W x 21.2H Centimeters","656.4 Grams"]

In [205]:
scraper=AutoScraper()
result=scraper.build(amazon_url,wanted_list)
print(result)

[]


In [206]:
scraper.get_result_similar(amazon_url,grouped=True)

{}

In [207]:
import csv
from autoscraper import AutoScraper

# Load the CSV file containing product links
product_links_file = './All Electronics.csv'

# Specify the wanted list of data to extract
wanted_list = ["₹599.00 ", "Electric kettle", "18.5L x 18.5W x 21.2H Centimeters", "656.4 Grams"]

# Create an instance of AutoScraper
scraper = AutoScraper()

# Create a function to scrape product details
def scrape_product_details(product_link):
    # Build the scraper for the product URL
    scraper.build(product_link, wanted_list)

    # Scrape the product details
    result = scraper.get_result_similar(product_link, grouped=True)

    # Extract the specific details from the result
    price = result['rule_6s8s']
    generic_name = result['rule_u3z3']
    dimensions = wanted_list[2]
    weight = wanted_list[3]

    return [price, generic_name, dimensions, weight]

# Create a function to save scraped data to a CSV file
def save_to_csv(data, output_file):
    with open(output_file, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Price', 'Generic Name', 'Dimensions', 'Weight'])
        writer.writerows(data)

# Initialize an empty list to store the scraped data
scraped_data = []

# Read the product links from the CSV file
with open(product_links_file, 'r') as file:
    reader = csv.reader(file)
    next(reader)  # Skip the header row
    for row in reader:
        product_link = row[0]
        product_details = scrape_product_details(product_link)
        scraped_data.append(product_details)

# Save the scraped data to a new CSV file
output_file = 'scraped_data.csv'
save_to_csv(scraped_data, output_file)

print("Scraping completed. The data has been saved to 'scraped_data.csv'.")


MissingSchema: Invalid URL 'Redmi 10 Power (Power Black, 8GB RAM, 128GB Storage)': No scheme supplied. Perhaps you meant http://Redmi 10 Power (Power Black, 8GB RAM, 128GB Storage)?

In [ ]:
import autoscraper
import csv

def scrape_amazon_products(csv_file_path, product_links):
  with open(csv_file_path, 'w', newline='') as csvfile:
    fieldnames = ['Price', 'Generic Name', 'Dimensions', 'Weight']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    for product_link in product_links:
      scraper = autoscraper.AutoScraper()
      result = scraper.build(product_link, wanted_list)
      data = {
        'Price': result['rule_hbsb'],
        'Generic Name': result['rule_q4do'],
        'Dimensions': result['rule_1000'],
        'Weight': result['rule_1001']
      }
      writer.writerow(data)

if __name__ == '__main__':
  csv_file_path = 'amazon-products.csv'
  product_links = []

  with open('./All Electronics.csv', 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
      product_links.append(row[0])

  scrape_amazon_products(csv_file_path, product_links)


MissingSchema: Invalid URL 'name': No scheme supplied. Perhaps you meant http://name?

In [209]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [210]:
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string
# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find("span", attrs={'class':'a-price-whole'}).string.strip()

    except AttributeError:

        try:
            # If there is some deal price
            price = soup.find("span", attrs={'id':'priceblock_dealprice'}).string.strip()

        except:
            price = ""

    return price

#Funtion to extract dimensions
def get_dimension(soup):
    try:
        dimension=soup.find("td",attr={'class':'a-size-base prodDetAttrValue"'}).string.strip()
    except AttributeError:
        dimension =" "
    return dimension

# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("span", attrs={'class':'a-size-base a-color-base'}).string.strip()
    
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""	

    return rating



In [216]:
if __name__ == '__main__':

    # add your user agent 
    HEADERS = ({'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36', 'Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    URL = "https://www.amazon.in/Redmi-Power-Black-128GB-Storage/dp/B09Y64H8VS/"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}
   
    # Loop for extracting product details from each link 
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.in" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
    

    
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)

In [212]:
amazon_df

,title,price,rating,reviews,availability


In [222]:
import requests
from bs4 import BeautifulSoup
import csv

# Function to scrape data from a single Amazon product page
def scrape_product_data(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Extract price
    price_tag = soup.find('span', class_='a-price-whole')
    price = price_tag.text if price_tag else 'N/A'
    
    # Extract product information
    product_info_tag = soup.find('table', id='productDetails_techSpec_section_1').find('td', class_='a-size-base prodDetAttrValue')
    product_info = product_info_tag.text if product_info_tag else 'N/A'
    
    # Extract item weight
    item_weight_tag = soup.find('table', id='productDetails_techSpec_section_1').find_all('td', class_='a-size-base prodDetAttrValue')[1]
    item_weight = item_weight_tag.text if item_weight_tag else 'N/A'
    
    return price, product_info, item_weight

# Read the file containing links
with open('./All Electronics.csv', 'r') as file:
    links = file.readlines()

# Scrape data for each link
data = []
for link in links:
    link = link.strip()  # Remove any leading/trailing whitespace or newline characters
    price, product_info, item_weight = scrape_product_data(link)
    data.append([link, price, product_info, item_weight])

# Display the scraped data
for d in data:
    print(f"Link: {d[0]}")
    print(f"Price: {d[1]}")
    print(f"Product Information: {d[2]}")
    print(f"Item Weight: {d[3]}")
    print("")

# Append the data to a CSV file
header = ['Link', 'Price', 'Product Information', 'Item Weight']
with open('scraped_data.csv', 'a', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(header)
    writer.writerows(data)

print("Scraped data has been appended to 'scraped_data.csv'.")


AttributeError: 'NoneType' object has no attribute 'find'

In [235]:
import requests
from bs4 import BeautifulSoup
import csv

# Function to scrape data from a single Amazon product page
def scrape_product_data(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Extract price
    price_tag = soup.find('span', class_='a-price-whole')
    price = price_tag.text if price_tag else 'N/A'
    
    # Extract product information
    product_info_table = soup.find('table', id='productDetails_techSpec_section_1')
    if product_info_table:
        product_info_rows = product_info_table.find_all('tr')
        product_info = []
        for row in product_info_rows:
            th = row.find('th', class_='a-color-secondary a-size-base prodDetSectionEntry')
            if th and th.text.strip() == 'Product Dimensions':
                td = row.find('td', class_='a-size-base prodDetAttrValue')
                if td:
                    product_info.append(td.text.strip())
    else:
        product_info = ['N/A']
    
    return price, product_info

# Read the file containing links
with open('./All Electronics.csv', 'r') as file:
    links = file.readlines()

# Scrape data for each link
data = []
for link in links:
    link = link.strip() 
    price, product_info = scrape_product_data(link)
    data.append([link, price, product_info])

# Display the scraped data
for d in data:
    print(f"Link: {d[0]}")
    print(f"Price: {d[1]}")
    print("Product Information:")
    for info in d[2]:
        print(info)
    print("")

# Append the data to a CSV file
header = ['Link', 'Price', 'Product Information']
with open('scraped_data.csv', 'a', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(header)
    writer.writerows(data)

print("Scraped data has been appended to 'scraped_data.csv'.")


Scraped data has been appended to 'scraped_data.csv'.


In [427]:
import requests
from bs4 import BeautifulSoup
import csv

# Function to scrape data from a single Amazon product page
def scrape_product_data(url):
    URL = url

    headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

    page = requests.get(URL, headers=headers)

    soup1 = BeautifulSoup(page.content, "html.parser")
    print("Soup-1",soup1)

    soup2 = BeautifulSoup(soup1.prettify(), "html.parser")
    title = soup2.find(id='productTitle').text.strip()
    price = soup2.find("span", attrs={"class": 'a-price-whole'}).string.strip()
    # Extract price
   
    
    return price, title

# Read the file containing links
with open('./All Electronics.csv', 'r') as file:
    links = file.readlines()

# Scrape data for each link
data = []
for link in links:
    link = link.strip() 
    price, title = scrape_product_data(link)
    print(link)
    print(price)
    data.append([link, price, title])

# Display the scraped data
for d in data:
    print(f"Link: {d[0]}")
    print(f"Price: {d[1]}")
    print("Product Information:")
    for info in d[2]:
        print(info)
    print("")

# Append the data to a CSV file
header = ['Link', 'Price', 'Title']
with open('scraped_data.csv', 'a', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(header)
    writer.writerows(data)

print("Scraped data has been appended to 'scraped_data.csv'.")


Soup-1 <!DOCTYPE html>

<!--[if lt IE 7]> <html lang="en-us" class="a-no-js a-lt-ie9 a-lt-ie8 a-lt-ie7"> <![endif]-->
<!--[if IE 7]>    <html lang="en-us" class="a-no-js a-lt-ie9 a-lt-ie8"> <![endif]-->
<!--[if IE 8]>    <html lang="en-us" class="a-no-js a-lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!-->
<html class="a-no-js" lang="en-us"><!--<![endif]--><head>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta charset="utf-8"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<title dir="ltr">Amazon.in</title>
<meta content="width=device-width" name="viewport"/>
<link href="https://images-na.ssl-images-amazon.com/images/G/01/AUIClients/AmazonUI-3c913031596ca78a3768f4e934b1cc02ce238101.secure.min._V1_.css" rel="stylesheet"/>
<script>

if (true === true) {
    var ue_t0 = (+ new Date()),
        ue_csm = window,
        ue = { t0: ue_t0, d: function() { return (+new Date() - ue_t0); } },
        ue_furl = "fls-eu.amazon.in",
        ue_mid = "A21TJRUUN

AttributeError: 'NoneType' object has no attribute 'text'

In [258]:
# import libraries 

from bs4 import BeautifulSoup
import requests
import time
import datetime

import smtplib

In [267]:
# Connect to Website and pull in data

URL = 'https://www.amazon.in/s?i=electronics&rh=n%3A21529675031%2Cp_36%3A1318503031&dc&fs=true&ds=v1%3AbbqLgg8UaXQRkx%2FMgvgKcAkKuQaTkLISXcyoe25rmrA&qid=1689442196&rnid=1318502031&ref=sr_nr_p_36_1'

headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

page = requests.get(URL, headers=headers)

soup1 = BeautifulSoup(page.content, "html.parser")
print("Soup-1",soup1)

soup2 = BeautifulSoup(soup1.prettify(), "html.parser")

# title = soup2.find(id='productTitle').get_text()

# price = soup2.find(id='priceblock_ourprice').get_text()


# print(title)
# print(price)

Soup-1 <!DOCTYPE html>
<html class="a-no-js" data-19ax5a9jf="dingo" lang="en-in"><!-- sp:feature:head-start -->
<head><script>var aPageStart = (new Date()).getTime();</script><meta charset="utf-8"/>
<!-- sp:end-feature:head-start -->
<!-- sp:feature:csm:head-open-part1 -->
<script type="text/javascript">var ue_t0=ue_t0||+new Date();</script>
<!-- sp:end-feature:csm:head-open-part1 -->
<!-- sp:feature:cs-optimization -->
<meta content="on" http-equiv="x-dns-prefetch-control"/>
<link crossorigin="" href="https://images-eu.ssl-images-amazon.com" rel="dns-prefetch"/>
<link crossorigin="" href="https://images-eu.ssl-images-amazon.com" rel="preconnect"/>
<link crossorigin="" href="https://m.media-amazon.com" rel="dns-prefetch"/>
<link crossorigin="" href="https://m.media-amazon.com" rel="preconnect"/>
<link crossorigin="" href="https://completion.amazon.com" rel="dns-prefetch"/>
<link crossorigin="" href="https://completion.amazon.com" rel="preconnect"/>
<!-- sp:end-feature:cs-optimization -

In [268]:
soup2 = BeautifulSoup(soup1.prettify(), "html.parser")
print(soup2)

<!DOCTYPE html>

<html class="a-no-js" data-19ax5a9jf="dingo" lang="en-in">
<!-- sp:feature:head-start -->
<head>
<script>
   var aPageStart = (new Date()).getTime();
  </script>
<meta charset="utf-8"/>
<!-- sp:end-feature:head-start -->
<!-- sp:feature:csm:head-open-part1 -->
<script type="text/javascript">
   var ue_t0=ue_t0||+new Date();
  </script>
<!-- sp:end-feature:csm:head-open-part1 -->
<!-- sp:feature:cs-optimization -->
<meta content="on" http-equiv="x-dns-prefetch-control"/>
<link crossorigin="" href="https://images-eu.ssl-images-amazon.com" rel="dns-prefetch"/>
<link crossorigin="" href="https://images-eu.ssl-images-amazon.com" rel="preconnect"/>
<link crossorigin="" href="https://m.media-amazon.com" rel="dns-prefetch"/>
<link crossorigin="" href="https://m.media-amazon.com" rel="preconnect"/>
<link crossorigin="" href="https://completion.amazon.com" rel="dns-prefetch"/>
<link crossorigin="" href="https://completion.amazon.com" rel="preconnect"/>
<!-- sp:end-feature:cs-opt

In [275]:
# title = soup2.find(id='productTitle').get_text().strip()
price = soup2.find_all("span", attrs={"class": 'a-size-base-plus a-color-base a-text-normal'})
description=soup2.find_all("span",attrs={"class":'c0118'})

# rating=soup2.find("span" ,attrs={"class":'a-size-base a-color-base'}).get_text().strip()
# reviews=soup2.find(id='acrCustomerReviewText').get_text().strip()
# discount=soup2.find("span",attrs={"class":'a-size-large a-color-price savingPriceOverride aok-align-center reinventPriceSavingsPercentageMargin savingsPercentage'}).get_text().strip()

# print(title)
# print(price)
print(description)
# print(reviews)
# print(discount)




[]


In [155]:
import csv
import requests
from bs4 import BeautifulSoup

# Function to scrape information from an Amazon product page
def scrape_amazon_product(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36",
        "Accept-Encoding": "gzip, deflate",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
        "DNT": "1",
        "Connection": "close",
        "Upgrade-Insecure-Requests": "1"
    }

    page = requests.get(url, headers=headers)
    soup1 = BeautifulSoup(page.content, "html.parser")
    soup2 = BeautifulSoup(soup1.prettify(), "html.parser")

    title = soup2.find(id='productTitle').get_text().strip()
    price = soup2.find("span", attrs={"class": 'a-price-whole'}).get_text().strip()
    rating = soup2.find("span", attrs={"class": 'a-size-base a-color-base'}).get_text().strip()
    reviews = soup2.find(id='acrCustomerReviewText').get_text().strip()
    discount = soup2.find("span", attrs={"class": 'a-size-large a-color-price savingPriceOverride aok-align-center reinventPriceSavingsPercentageMargin savingsPercentage'}).get_text().strip()

    return title, price, rating, reviews, discount


# Read the CSV file containing the links
csv_file = './40.csv'

with open(csv_file, 'r') as file:
    reader = csv.reader(file)
    next(reader)  # Skip the header row
    for row in reader:
        product_link = row[0]
        
        # Scrape information from the Amazon product page
        title, price, rating, reviews, discount = scrape_amazon_product(product_link)
        print(title)
        # Write the extracted information to a new CSV file
        with open('Electronics_product_data.csv', 'a', newline='') as outfile:
            writer = csv.writer(outfile)
            writer.writerow([product_link, title, price, rating, reviews, discount])
        print("Data extracted and saved for:", product_link)


OnePlus Bullets Z2 Bluetooth Wireless in Ear Earphones with Mic, Bombastic Bass - 12.4 mm Drivers, 10 Mins Charge - 20 Hrs Music, 30 Hrs Battery Life, IP55 Dust and Water Resistant (Magico Black)
Data extracted and saved for: https://www.amazon.in/Oneplus-Bluetooth-Wireless-Earphones-Bombastic/dp/B09TVVGXWS/ref=sr_1_6?qid=1679133649&s=electronics&sr=1-6


AttributeError: 'NoneType' object has no attribute 'get_text'

In [229]:
import csv
import requests
from bs4 import BeautifulSoup
import os

# Function to scrape information from an Amazon product page
def scrape_amazon_product(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36",
        "Accept-Encoding": "gzip, deflate",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
        "DNT": "1",
        "Connection": "close",
        "Upgrade-Insecure-Requests": "1"
    }

    page = requests.get(url, headers=headers)
    soup1 = BeautifulSoup(page.content, "html.parser")
    soup2 = BeautifulSoup(soup1.prettify(), "html.parser")

    title = soup2.find(id='productTitle').get_text().strip()
    price = soup2.find("span", attrs={"class": 'a-price-whole'}).get_text().strip()
    rating = soup2.find("span", attrs={"class": 'a-size-base a-color-base'}).get_text().strip()
    reviews = soup2.find(id='acrCustomerReviewText').get_text().strip()
    discount = soup2.find("span", attrs={"class": 'a-size-large a-color-price savingPriceOverride aok-align-center reinventPriceSavingsPercentageMargin savingsPercentage'}).get_text().strip()

    return title, price, rating, reviews, discount


# Read the CSV file containing the links
csv_file = '40.csv'
new_csv_file = 'Electronics_product_data.csv'

with open(csv_file, 'r') as file:
    reader = csv.reader(file)
    header = next(reader)  # Get the header row
    links = list(reader)  # Store the remaining rows as a list

for link in links:
    product_link = link[0]

    # Scrape information from the Amazon product page
    title, price, rating, reviews, discount = scrape_amazon_product(product_link)
    print(title)

    # Write the extracted information to a new CSV file
    with open(new_csv_file, 'a', newline='') as outfile:
        writer = csv.writer(outfile)
        writer.writerow([product_link, title, price, rating, reviews, discount])
    print("Data extracted and saved for:", product_link)

    # Delete the processed link from the old CSV file
    with open(csv_file, 'w', newline='') as outfile:
        writer = csv.writer(outfile)
        writer.writerow(header)  # Rewrite the header row
        writer.writerows(links)  # Rewrite the remaining rows

    # Remove the old CSV file if it is empty
    if os.stat(csv_file).st_size == 0:
        os.remove(csv_file)


AttributeError: 'NoneType' object has no attribute 'get_text'

Loop

In [257]:
import requests
from bs4 import BeautifulSoup

# Function to scrape information from an Amazon product page
def scrape_amazon_product(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36",
        "Accept-Encoding": "gzip, deflate",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
        "DNT": "1",
        "Connection": "close",
        "Upgrade-Insecure-Requests": "1"
    }

    page = requests.get(url, headers=headers)
    soup1 = BeautifulSoup(page.content, "html.parser")
    soup2 = BeautifulSoup(soup1.prettify(), "html.parser")

    title = soup2.find(id='productTitle').get_text().strip()
    price = soup2.find("span", attrs={"class": 'a-price-whole'}).get_text().strip()
    rating = soup2.find("span", attrs={"class": 'a-size-base a-color-base'}).get_text().strip()
    reviews = soup2.find(id='acrCustomerReviewText').get_text().strip()
    discount = soup2.find("span", attrs={"class": 'a-size-large a-color-price savingPriceOverride aok-align-center reinventPriceSavingsPercentageMargin savingsPercentage'}).get_text().strip()

    return title, price, rating, reviews, discount


# List of product URLs
urls = [
    'https://www.amazon.in/Samsung-Mystique-Storage-Purchased-Separately/dp/B09TWGDY4W/ref=sr_1_7?qid=1679133649&s=electronics&sr=1-7',
'https://www.amazon.in/OnePlus-Nord-Black-128GB-Storage/dp/B09WQY65HN/ref=sr_1_8?qid=1679133649&s=electronics&sr=1-8'
'https://www.amazon.in/Syska-Power-Wheel-Extension-SSK-PW-0301/dp/B01KWX2JGM/ref=sr_1_496?qid=1679133679&s=electronics&sr=1-496',
'https://www.amazon.in/Oppo-Bluetooth-Wireless-Earbuds-Charging/dp/B0B1ZYJK71/ref=sr_1_497?qid=1679133679&s=electronics&sr=1-497',
'https://www.amazon.in/Zebronics-Zeb-Sound-Playback-Controls-Assistant/dp/B098XR32X2/ref=sr_1_498?qid=1679133679&s=electronics&sr=1-498',
'https://www.amazon.in/Boat-Rockerz-450-Lightweight-Compatibility/dp/B08667772G/ref=sr_1_499?qid=1679133679&s=electronics&sr=1-499',
'https://www.amazon.in/boAt-Immortal-Wireless-Earbuds-Signature/dp/B0BKZLW5T5/ref=sr_1_500?qid=1679133679&s=electronics&sr=1-500',
'https://www.amazon.in/Essentials-G11-Earphone-Carrying-Earphones/dp/B07DKZCZ89/ref=sr_1_501?qid=1679133679&s=electronics&sr=1-501',
'https://www.amazon.in/Nokia-105-Single-Wireless-Charcoal/dp/B09YDFKJF8/ref=sr_1_502?qid=1679133679&s=electronics&sr=1-502',
]

# Iterate over the URLs and scrape information
for url in urls:
    title, price, rating, reviews, discount = scrape_amazon_product(url)
    print("Title:", title)
    print("Price:", price)
    print("Reviews:", reviews)
    print("Discount:", discount)
    print()


AttributeError: 'NoneType' object has no attribute 'get_text'

In [519]:

from autoscraper import AutoScraper

In [520]:

amazon_url="https://www.amazon.in/Apple-iPhone-14-128GB-Blue/dp/B0BDK62PDX/ref=sr_1_1_sspa?keywords=iphone&qid=1688608267&sr=8-1-spons&sp_csd=d2lkZ2V0TmFtZT1zcF9hdGY&psc=1"

wanted_list=["₹70,999","Apple iPhone 14 (128 GB) - Blue","5.78 inches (146.7 mm)","2.82 inches (71.5 mm)","0.31 inch (7.80 mm)","6.07 ounces (172 grams)"]

In [521]:
scraper=AutoScraper()
result=scraper.build(amazon_url,wanted_list)
print(result)

['₹70,999', 'Apple iPhone 14 (128 GB) - Blue']


Preprocessing

In [526]:
import csv

# Define the column indices in the CSV file
PRODUCT_NAME_INDEX = 0
CATEGORY_INDEX = 1
SELLING_PRICE_INDEX = 2
PRODUCT_SPEC_INDEX = 3
SHIPPING_WEIGHT_INDEX = 4
IMAGE_DATA_INDEX = 5

# Read the CSV file
with open('./sample_data.csv', 'r') as file:
    reader = csv.reader(file)
    next(reader)  # Skip the header row

    # Create a new list to store the filtered rows
    filtered_rows = []

    # Iterate over each row in the CSV file
    for row in reader:
        product_spec = row[PRODUCT_SPEC_INDEX]

        # Check if the product specification data is present
        if product_spec:
            filtered_rows.append(row)

# Write the filtered rows to a new CSV file
with open('filtered_file_10.xls', 'w', newline='') as file:
    writer = csv.writer(file)

    # Write the header row
    writer.writerow(['Product Name', 'Category', 'Selling Price', 'Product Specification', 'Shipping Weight', 'Image Data'])

    # Write the filtered rows
    writer.writerows(filtered_rows)

print("Preprocessing completed. Filtered data has been saved to 'filtered_file.xls'.")


Preprocessing completed. Filtered data has been saved to 'filtered_file.xls'.


In [310]:
from bs4 import BeautifulSoup
import requests

# Function to extract Product Title
def get_title(soup):
	
	try:
		# Outer Tag Object
		title = soup.find("span", attrs={"id":'productTitle'})

		# Inner NavigatableString Object
		title_value = title.string

		# Title as a string value
		title_string = title_value.strip()

		# # Printing types of values for efficient understanding
		# print(type(title))
		# print(type(title_value))
		# print(type(title_string))
		# print()

	except AttributeError:
		title_string = ""	

	return title_string

# Function to extract Product Price
def get_price(soup):

	try:
		price = soup.find("span", attrs={'id':'priceblock_ourprice'}).string.strip()

	except AttributeError:

		try:
			# If there is some deal price
			price = soup.find("span", attrs={'id':'priceblock_dealprice'}).string.strip()

		except:		
			price = ""	

	return price

# Function to extract Product Rating
def get_rating(soup):

	try:
		rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()
		
	except AttributeError:
		
		try:
			rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
		except:
			rating = ""	

	return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
	try:
		review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()
		
	except AttributeError:
		review_count = ""	

	return review_count

# Function to extract Availability Status
def get_availability(soup):
	try:
		available = soup.find("div", attrs={'id':'availability'})
		available = available.find("span").string.strip()

	except AttributeError:
		available = "Not Available"	

	return available	


if __name__ == '__main__':

	# Headers for request
	HEADERS = ({'User-Agent':
	            'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
	            'Accept-Language': 'en-US'})

	# The webpage URL
	URL = "https://www.amazon.com/s?k=playstation+4&ref=nb_sb_noss_2"
	
	# HTTP Request
	webpage = requests.get(URL, headers=HEADERS)

	# Soup Object containing all data
	soup = BeautifulSoup(webpage.content, "lxml")

	# Fetch links as List of Tag Objects
	links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})

	# Store the links
	links_list = []

	# Loop for extracting links from Tag Objects
	for link in links:
		links_list.append(link.get('href'))


	# Loop for extracting product details from each link 
	for link in links_list:

		new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)

		new_soup = BeautifulSoup(new_webpage.content, "lxml")
		
		# Function calls to display all necessary product information
		print("Product Title =", get_title(new_soup))
		print("Product Price =", get_price(new_soup))
		print("Product Rating =", get_rating(new_soup))
		print("Number of Product Reviews =", get_review_count(new_soup))
		print("Availability =", get_availability(new_soup))
		print()
		print()

Product Title = 
Product Price = 
Product Rating = 
Number of Product Reviews = 
Availability = Not Available


Product Title = 
Product Price = 
Product Rating = 
Number of Product Reviews = 
Availability = Not Available


Product Title = 
Product Price = 
Product Rating = 
Number of Product Reviews = 
Availability = Not Available


Product Title = 
Product Price = 
Product Rating = 
Number of Product Reviews = 
Availability = Not Available


Product Title = 
Product Price = 
Product Rating = 
Number of Product Reviews = 
Availability = Not Available


Product Title = 
Product Price = 
Product Rating = 
Number of Product Reviews = 
Availability = Not Available


Product Title = 
Product Price = 
Product Rating = 
Number of Product Reviews = 
Availability = Not Available


Product Title = 
Product Price = 
Product Rating = 
Number of Product Reviews = 
Availability = Not Available


Product Title = 
Product Price = 
Product Rating = 
Number of Product Reviews = 
Availability = Not Avai